# Patent Layer Agent

## Imports

In [ ]:
import os
from dotenv import load_dotenv
import time
import requests

from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph, MessagesState, END
from langgraph.checkpoint.memory import MemorySaver

from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

import langchain_core

In [ ]:
load_dotenv()

In [ ]:
SERP_API_KEY = os.getenv("SERP_API_KEY")
SERP_API_KEY

In [ ]:
openai_api = os.getenv("OPENAI_API_KEY")
langsmith_api = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

## Load Documents

In [ ]:
file_path = os.path.join(os.path.dirname(os.getcwd()), 'docs', 'knowledge_base')
file_path

In [ ]:
pdf_list = [f for f in os.listdir(file_path) if f.endswith('.pdf')]
pdf_list

In [ ]:
pdf_docs = {}
for i in range(len(pdf_list)):
    loader = PyPDFLoader(os.path.join(file_path, pdf_list[i]))
    pages = loader.load()
    pdf_docs[pdf_list[i]] = pages

pdf_docs

In [ ]:
pdf_docs['top-tips-for-o.pdf'][2].page_content

## Split documents into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [ ]:
pdf_splits = {}
for p in pdf_docs:
    pdf_splits[p] = text_splitter.split_documents(pdf_docs[p])

pdf_splits

In [ ]:
len(pdf_docs['top-tips-for-o.pdf']), len(pdf_splits['top-tips-for-o.pdf'])

## Embed chunks and save in vector store

In [ ]:
persist_directory = os.path.join(os.path.dirname(os.getcwd()), 'data')
persist_directory

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
vector_store = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

In [ ]:
vector_store._collection.count()

In [ ]:
id_list = []
for s in pdf_splits:
    ids = vector_store.add_documents(pdf_splits[s])
    id_list.extend(ids)

In [ ]:
id_list[0], len(id_list)

In [ ]:
vector_store.get(id_list[0])

In [ ]:
vector_store._collection.count()

## Querying the data base

In [ ]:
results = vector_store.similarity_search_with_score(
    "What are the 3 most important things to know when submitting a patent application?",
    k=5
)

results

In [ ]:
print(results[0][0].page_content)

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
results = vectordb.similarity_search_with_score(
    "What are the 3 most important things to know when submitting a patent application?",
    k=5
)

results

In [ ]:
print(results[0][0].page_content)

## Retrieval

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 20},
)

retriever

In [ ]:
retriever.invoke("What are the 3 most important things to know when submitting a patent application?")

## Q&A chat model

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

In [ ]:
prompt_template = """
    # Persona
    You are a patent lawyer. You are an expert in responding to Office Actions in a way that complies with the \
    MANUAL OF PATENT EXAMINING PROCEDURE. You have a deep knowledge and understanding in the patent rules and laws, \
    and you know how to respond to claim rejections either by making amendments to the claims or by making \
    remarks to dispute the rejection.

    # Task
    You will get a Patent Application document, and an Office Action document from the examiner of the \
    United States Patent and Trademark Office. Your job is to respond to all claim rejections in the best way \
    possible. You can amend a claim, add a remark, dispute the rejection, remove claim or unite two claims. \
    You will create a new document of 'Response to Office Action' in a formal way that is acceptable by the \
    United States Patent and Trademark Office.
    You will create the following sections one at a time:
    - Title
    - Amendment
    - Amendments to the specification
    - Amendments to the claims
    - Remarks
    - Conclusion

    Do NOT write it all in a single output, write each section individually. After you write a section ask the user \
    if it is good and wait for a reply before you continue. If the user approves send the section to make.com \
    using the sendPatentRequest tool. Then go on to write next section.

    # Action Steps
    - The user will give you a Patent application documents and an Office Action document.

    - Make sure you have all the information needed to respond to the Office Action, \
        and ask clarifying question if you need more information.

    - Write the Title and ask the user if it is good. Wait for a response before you continue. \
        If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the title to make.com in the 'invention_title' field of the schema.

    - Write the Amendment section with this template:
        "Commissioner for Patents
        P.O. Box 1450
        Alexandria, VA 22313-1450

        Sir:
            This communication is in reply to the Office Action dated {date}, and issued by the \
                United States Patent and Trademark Office in connection with the above-identified Patent Application."
      Fill in the the date variable with the date of the Office Action document, if you can't find the date ask the user for it. \
      Wait for a response before you continue. If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the text of this section to make.com in the 'amendment' field of the schema.

    - Write the Amendments to the specification section and ask the user if it is good. Wait for a response before you continue. \
        If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the text of this section to make.com \
        in the 'amendments_to_specification' field of the schema.

    - Write the Amendments to the claims section and ask the user if it is good. Wait for a response before you continue. \
        If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the text of this section to make.com \
        in the 'amendments_to_claims' field of the schema.

    - Write the Remarks section and ask the user if it is good. Wait for a response before you continue. \
        If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the text of this section to make.com in the 'remarks' field of the schema.

    - Write the Conclusion section and ask the user if it is good. Wait for a response before you continue. \
        If the user asks for adjustments, rewrite this section. Otherwise, proceed.

    - Use the sendPatentRequest tool to send the text of this section to make.com in the 'conclusion' field of the schema.

    # Examples
    Here are some examples of Office Actions and their corresponding responses:
    - Office Action 1:
            DETAILED ACTION Notice of Pre-AIA or AIA Status
        1. The present application, filed on or after March 16, 2013, is being examined under the first inventor to file provisions of the AIA.
        Information Disclosure Statement
        2. The information disclosure statements (IDS) submitted on 08/24/2022, 08/06/2023 and 12/26/2023 are acknowledged. The submission is in compliance with the provisions of 37 CFR 1.97. Accordingly, the information disclosure statements have been considered by the examiner. See attached copies of PTO-1449.
        Specification
        The disclosure is objected to because of the following informalities: The use of the terms Carbopol 974P NF (Carbomer Homopolymer Type B), Carbopol 971P NF (Carbomer Homopolymer Type A), Carbopol 934P NF (Carbomer 934P), Carbopol 71 G, Carbopol 980 NF (Carbomer Homopolymer Type C), Carbopol 981 NF, Carbopol 5964 EP, Carbopol 940 NF, Carbopol 941 NF, Carbopol 1342 NF, Carbopol 934 NF, Carbopol ETD 2020 NF, Carbopol Ultrez 10 NF, Carbopol TR-1 NF, Carbopol TR-2 NF, which are trade names or marks used in commerce, have been noted in this application. The term should be accompanied by the generic terminology; furthermore, the term should be capitalized wherever it appears or, where appropriate, include a proper symbol indicating use in commerce such as TM, SM, or R following the term.
        Although the use of trade names and marks used in commerce (i.e., trademarks, service marks, certification marks, and collective marks) are permissible in patent applications, the proprietary nature of the marks should be respected and every effort
        made to prevent their use in any manner which might adversely affect their validity as
        commercial marks.
        Appropriate correction is required.
        Claim Rejections - 35 USC § 112
        3. The following is a quotation of 35 U.S.C. 112(b):
        (b) CONCLUSION.-The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the inventor or a joint inventor regards as the invention.
        The following is a quotation of 35 U.S.C. 112 (pre-AIA), second paragraph: The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the applicant regards as his invention.
        Claim 4 is rejected under 35 U.S.C. 112(b) or 35 U.S.C. 112 (pre-AIA), second paragraph, as being indefinite for failing to particularly point out and distinctly claim the subject matter which the inventor or a joint inventor (or for applications subject to pre- AIA 35 U.S.C. 112, the applicant), regards as the invention.

        Claim 4 contains the trademark/trade name Carbopol. Where a trademark or trade name is used in a claim as a limitation to identify or describe a particular material or product, the claim does not comply with the requirements of 35 U.S.C. 112(b) or 35 U.S.C. 112 (pre-AIA), second paragraph. See Ex parte Simpson, 218 USPQ 1020 (Bd. App. 1982). The claim scope is uncertain since the trademark or trade name cannot be used properly to identify any particular material or product. A trademark or trade name is used to identify a source of goods, and not the goods themselves. Thus, a trademark or trade name does not identify or describe the goods associated with the trademark or trade name. In the present case, the trademark/trade name is used to
        identify/describe specific polyacrylic polymers and, accordingly, the identification/description is indefinite.
        Claim Rejections - 35 USC § 102
        4. The following is a quotation of the appropriate paragraphs of 35 U.S.C. 102 that
        form the basis for the rejections under this section made in this Office action:
        A person shall be entitled to a patent unless -
        (a)(1) the claimed invention was patented, described in a printed publication, or in public use, on sale, or otherwise available to the public before the effective filing date of the claimed invention.
        (a)(2) the claimed invention was described in a patent issued under section 151, or in an application for patent published or deemed published under section 122(b), in which the patent or application, as the case may be, names another inventor and was effectively filed before the effective filing date of the claimed invention.
        Claims 1-11 are rejected under 35 U.S.C. 102(a)(1) as being anticipated by Nangia et al. (US 9744137 B2).
        Claims 1-3 and 6-8, Nangia et al. teach:
        Pharmaceutical compositions (Abs), refer to pellets, particles, microparticles, or minitablets or any combination thereof (Col. 16, lines 37-40).

        Certain delayed release (DR) pellets may be coated with EUDRAGIT® L30D-55, which dissolves at about pH 5.5-6.0, i.e., in the upper intestines. (Col. 23, line 15-17). The composition contains a core and an enteric coating surrounding the core. (Col. 1, line 15-17).
        The cores or starting seeds can be, sugar spheres or spheres made from microcrystalline cellulose or silicon dioxide or various salt crystals such as sodium citrate. (Col 20, lines 63-67), which are hydrophilic components. The immediate release composition may contain one or more binders to give the pellets cohesiveness. (col. 20, lines 17-18).
        In certain embodiments, at least a portion of the pellets are individually coated with both the bioadhesive polymer and the enteric coating. (Col 14, line 35-37).
        In some embodiments, an unbanded capsule is sealed with a sufficient amount of sealing composition. Such sealing compositions comprise a film-forming material, a plasticizer
        and a filler. The sealed capsule may be further coated with at least one other coating as described herein (col 40, line 25-29).
        In another aspect, a pharmaceutical composition comprises suitable pharmaceutically acceptable polymeric carriers include hydroxypropyl methyl cellulose (HPMC), sodium carboxymethyl cellulose (Na-CMC), (col 25, lines 58-61), polyvinylpyrrolidone (PVP), polyethylene glycol (PEG), (col. 26, lines 1-2). Preferred polymeric carriers are one or more of polyvinylpyrrolidone (PVP), block co-polymers of ethylene oxide and propylene oxide (PEO), and polyethylene glycol (PEG), wherein a more preferred polymeric carrier is polyvinylpyrrolidone (PVP) having an average molecular weight of about 2,500 to about 3,000,000. A most preferred polymeric carrier is polyvinylpyrrolidone (PVP) having an average molecular weight of from about 10,000 to about 450,000. The more preferred polymeric carrier is hydroxypropyl methyl cellulose (HPMC) having an average molecular weight of about 2,500 to about 100,000. A most preferred polymeric carrier is hydroxypropyl methyl cellulose having an average molecular weight of from about 5,000 to about 50,000. (Col. 26, lines 5-19). They are examples of channel forming agents and they include low molecular weight from
        as low as 2500 Dalton. Dalton unit (col. 54, line 67).
        The modification to the polymer or monomer can performed in the coupling reaction with a crosslinker, carbodiimide (col. 58, line 26-42). The dosage form matrix can be made by procedures known to the polymer art with a cross-linking agent. The step is repeated for each successive layer. Representative polymers suitable for manufacturing the dosage form include olefin and vinyl polymers, condensation polymers, carbohydrate polymers, and silicon polymers as represented by poly(ethylene), poly (propylene), poly(vinylacetate), poly(methyl acrylate), poly(isobutyl methacrylate), poly(alginate), poly(amide), and poly(silicone). (col 39, lines 51-67).
        With regard to claim 4, in certain embodiments, the bioadhesive layer comprises one or more polymeric materials selected from polyvinyl alcohols, Carbopols, (col. 7, lines 13, 15, 20, 23).

        With regard to claim 5, "Bioadhesion" is defined as the ability of a material to adhere to a biological tissue for an extended period of time. Polycarbophils and acrylic acid polymers usually have the best adhesive properties. (col 47, lines 1-2 & 10-11).
        With regard to claim 9, the term "particle" refers to particles that comprise topiramate or a pharmaceutically acceptable salt thereof in any suitable size and shape. Typically, particles have a diameter in the range of about 1 micron to about 300 microns. Examples of different particle shapes include rods, granules, planar structures and other regular or irregular shapes. (Col 16, lines 23-27), so products comprise microparticles of different diameters.
        With regard to claim 10, in other embodiments, the topiramate is micronized and the pellet size is less than 5 microns. Particles are formulated to these sizes in
        order to enable high drug loading when needed. (col. 44, lines 62-65), so microparticles are of substantially similar diameter.
        With regard to claim 11, Active pellets were prepared by extrusion
        spheronization.
        Topiramate was blended with Emcocel 90M, Hydroxypropylcellulose EF Pharm, dibasic
        sodium phosphate, sodium lauryl sulfate, and Povidone K-30 and wet granulated by addition of purified water while mixing in a Hobart mixer. The granulation was then extruded into rods using a Caleva (col. 71, line 62-67).
        Conclusion
        5. No claim is allowed at this time.
        6. Any inquiry concerning this communication or earlier communications from the examiner should be directed to NGOC-ANH THI NGUYEN whose telephone number is (571)270-0867. The examiner can normally be reached Monday - Friday 8:00 am.

        Examiner interviews are available via telephone, in-person, and video
        conferencing using a USPTO supplied web-based collaboration tool. To schedule an interview, applicant is encouraged to use the USPTO Automated Interview Request (AIR) at http://www.uspto.gov/interviewpractice.
        If attempts to reach the examiner by telephone are unsuccessful, the examiner's supervisor, Robert A Wax can be reached on 571-272-0623. The fax phone number for the organization where this application or proceeding is assigned is 571-273-8300.
        Information regarding the status of published or unpublished applications may be obtained from Patent Center. Unpublished application information in Patent Center is available to registered users. To file and manage patent submissions in Patent Center, visit: https:// patentcenter.uspto.gov. Visit https://www.uspto.gov/patents/apply/patent- center for more information about Patent Center and https://www.uspto.gov/patents/docx for information about filing in DOCX format. For additional questions, contact the Electronic Business Center (EBC) at 866-217-9197 (toll-free). If you would like assistance from a USPTO Customer Service
        Representative, call 800-786-9199 (IN USA OR CANADA) or 571-272-1000.

    - Response to Office Action 1:
            AMENDMENT

        Commissioner for Patents
        P.O. Box 1450
        Alexandria, VA 22313-1450

        Sir:
        This communication is in reply to the Office Action dated September 25, 2024, and issued by the United States Patent and Trademark Office in connection with the above-identified Patent Application.

        AMENDMENTS TO THE SPECIFICATION

        Kindly amend the specification as follows:
        Kindly replace the paragraphs at page 3, lines 1-12, with the following amended paragraphs:
        According to some demonstrative embodiments, the bioadhesive material may be selected from selected from one or more of the following groups of materials:
        Polyanionic polymers selected from the group including polycarbophil USP, cross-linked polyacrylic acid polymers such as (carbomer homopolymer) (commercially available under various trade names including Carbopol® 974P NF (Carbomer Homopolymer Type B), Carbopol® 971P NF (Carbomer Homopolymer Type A), Carbopol® 934P NF (Carbomer 934P), Carbopol® 71 G, Carbopol® 980 NF (Carbomer Homopolymer Type C), Carbopol® 981 NF, Carbopol® 5964 EP, Carbopol® 940 NF, Carbopol® 941 NF, Carbopol® 1342 NF, Carbopol® 934 NF, Carbopol® ETD 2020 NF, Carbopol® Ultrez 10 NF, Carbopol® TR-1 NF, Carbopol® TR-2 NF, or a combinations thereof.
        b. Polycationic polymers selected from the group including cationic starch, cationic polyvinyl alcohol, cationic polysaccharide and/or cationic gum.

        Kindly replace the paragraphs at page 6, lines 9-24, with the following amended paragraphs:
        I. Polymers of acrylic acid crosslinked with allyl sucrose or allyl pentaerythritol (also known as Carbopol homopolymers).
        Cross-linked polyacrylic acid polymers (carbomer homopolymers) formed by polymers of acrylic acid crosslinked with allyl sucrose or allyl pentaerythritol (commercially available under trade names including various Carbopol® homopolymers).
        II. Polymers of acrylic acid and C10-C30 alkyl acrylate crosslinked with allyl pentaerythritol (also known as Carbopol copolymers).
        (commercially available under trade names including various Carbopol® copolymers).
        III. Carbomer homopolymer or copolymer that contains a block copolymer of polyethylene glycol and a long chain alkyl acid ester (also known as Carbopol®) interpolymers).
        According to some demonstrative embodiments, the bioadhesive may be selected from one or more of the following groups of materials:
        Polyanionic polymers selected from the group including Polyanionic polymers selected from the group including polycarbophil AA-1 USP, cross-linked polyacrylic acid polymers such as Carbopol 974P NF (Carbomer Homopolymer Type B), Carbopol 971P NF (Carbomer Homopolymer Type A), Carbopol 934P NF (Carbomer 934P), Carbopol 71 G, Carbopol 980 NF (Carbomer Homopolymer Type C), Carbopol 981 NF, Carbopol 5964 EP, Carbopol 940 NF, Carbopol 941 NF, Carbopol 1342 NF, Carbopol 934 NF, Carbopol ETD 2020 NF, Carbopol Ultrez 10 NF, Carbopol TR-1 NF, Carbopol TR-2 NF, or a combinations thereof
        polycarbophil AA-1 USP, cross-linked polyacrylic acid polymers (carbomer homopolymer) (commercially available under various trade names such as Carbopol® 974P NF (Carbomer Homopolymer Type B), Carbopol® 971P NF (Carbomer Homopolymer Type A), Carbopol® 934P NF (Carbomer 934P), Carbopol® 71 G, Carbopol® 980 NF (Carbomer Homopolymer Type C), Carbopol® 981 NF, Carbopol® 5964 EP, Carbopol® 940 NF, Carbopol® 941 NF, Carbopol® 1342 NF, Carbopol® 934 NF, Carbopol® ETD 2020 NF, Carbopol® Ultrez 10 NF, Carbopol® TR-1 NF, Carbopol® TR-2 NF), or a combination thereof.

        AMENDMENTS TO THE CLAIMS
        Kindly amend the claims as follows:
        1. (Currently Amended) A microparticle for the administration of a pharmaceutical composition at the upper gastrointestinal tract comprising:
        a core comprising
        at least one pharmaceutical composition[[;]] and
        at least one excipient; and
        a first coating layer comprising
        a bioadhesive material[[;]] ,
        optionally, a channel forming agent[[;]] ,
        a film forming polymer[[;]] , and
        a cross-linker,
        wherein said cross-linker interacts with said film forming polymer upon exposure to an aqueous environment.
        2. (Original) The microparticle of claim 1, wherein said core further comprises a hydrophilic component and a binder.
        3. (Original) The microparticle of claim 1, wherein said first layer further comprises channel forming agent.
        4. (Currently Amended) The microparticle of claim 1, wherein said bioadhesive material is selected from selected from one or more of the following groups of materials:
        [[c]] Polyanionic polymers selected from the group including polycarbophil USP, Carbopol 974P NF (Carbomer Homopolymer Type B), Carbopol 971P NF (Carbomer Homopolymer Type A), Carbopol 934P NF (Carbomer 934P), Carbopol 71 G, Carbopol 980 NF (Carbomer Homopolymer Type C), Carbopol 981 NF, Carbopol 5964 EP, Carbopol 940 NF, Carbopol 941 NF, Carbopol 1342 NF, Carbopol 934 NF, Carbopol ETD 2020 NF, Carbopol Ultrez 10 NF, Carbopol TR-1 NF, Carbopol TR-2 NF, cross-linked polyacrylic acid polymers (carbomer homopolymer), or a combinations thereof;
        [[d]] Polycationic polymers selected from the group including cationic starch, cationic polyvinyl alcohol, cationic polysaccharide, and/or cationic gum.
        5. (Original) The microparticle of claim 4, wherein said bioadhesive material comprises polycarbophil USP.
        6. (Original) The microparticle of claim 1, wherein said cross linker is a composition which allows for the in-situ cross linking of the film forming polymer and is selected from the group including Caffeic acid, tannic acid, chicoric acid, carbodiimide, genipin, Gallic acid, boric acid and sodium borate (borax).
        7. (Original) The microparticle of claim 1, wherein said channel forming agent comprises a polymer selected from the group including low molecular weight Polyethylene glycol (PEG), low molecular weight PolyEthylene oxide (PEO), low molecular weight Polyvinylpyrrolidone (PVP), low molecular weight Polyvinyl alcohol (PVA), low molecular weight Sodium Carboxymethyl cellulose (Na-CMC), low molecular weight Hydroxyethyl cellulose (HEC).
        8. (Original) The microparticle of claim 1, wherein said channel forming agent is a low molecular weight PVP below 30 Kilo Dalton.
        9. (Original) A product comprising the microparticle of claim 1, wherein said product comprises microparticles of different diameters.
        10. (Original) A product comprising the microparticle of claim 1, wherein said product comprises microparticles of substantially similar diameter.
        11. (Original) A process for the preparation of the microparticle of claim 1, wherein said core is prepared by granulation or extrusion.


        REMARKS
        The present response is intended to be fully responsive to all points of rejection raised by the Examiner in the instant Office Action, and is believed to place the application in condition for allowance. Favorable reconsideration and allowance of the application is respectfully requested.
        Amendments to the Specification
        The specification is objected to over the use of “Carbopol 974P NF, etc.” as a trade name. Applicant has now amended the Specification to specify generic chemical names (e.g., “carbomer homopolymer”), while indicating that “Carbopol®” is a registered mark.
        Claim Rejections under 35 U.S.C. §112
        Claim 4 is rejected under 35 U.S.C. §112 as being indefinite and as amended herewith to recite “cross-linked polyacrylic acid polymer (carbomer homopolymer)” in place of “Carbopol® 974P NF” to refer to the substance rather than its brand/trade name. Reconsideration of claim 4 under 35 U.S.C. §112 is respectfully requested.
        Claim Rejections under 35 U.S.C. §102
        Claims 1-11 are rejected under 35 U.S.C. §102 as being anticipated by U.S. Patent No. 9,744,137 (hereinafter “Nangia”). Applicant respectfully traverses.
        Anticipation requires that a single prior art reference disclose each and every element of the claimed invention, explicitly or inherently. MPEP § 2131 states:
        “A claim is anticipated only if each and every element as set forth in the claim is found, either expressly or inherently described, in a single prior art reference.” [Emphasis Added]
        See also Net MoneyIN v. Verisign, 545 F.3d 1359 (Fed. Cir. 2008).
        Applicant respectfully traverses the anticipation rejection for at least the following reasons:
        The Examiner's combination of disparate passages that do not collectively teach a “single first coating layer” containing all the recited elements

        Claim 1 specifically recites a microparticle whose “first coating layer” contains a bioadhesive polymer, a film-forming polymer, a cross-linker, and optionally a channel-forming agent. Nothing in Nangia teaches or inherently discloses a single layer combining all these components such that the cross-linker remains substantially unreacted until contact with fluids in the upper GI.
        MPEP § 2131.01 further notes:
        “…the burden is on the examiner to show that all elements of the claimed invention are disclosed within the four corners of a single prior art document…”
        And see for example, “In re Robertson, 169 F.3d 743, 745 (Fed. Cir. 1999).
        While Nangia references “bioadhesive polymers” (Carbopols) and also references crosslinkers (e.g., carbodiimides for polymer modification), it does not teach the arrangement or the in situ crosslinking mechanism recited.

        The in situ crosslinking triggered in an aqueous environment
        A critical element of Claim 1 is that the cross-linker in the first coating layer reacts with the film-forming polymer “upon exposure to an aqueous environment,” thereby forming a crosslinked matrix. Nangia's discussion of crosslinkers (col. 58, lines 26-42) appears tied to manufacturing or pre-processing steps. There is no disclosure that the crosslinker is held within a single layer to be triggered in situ in the GI.
        Nangia's text regarding “carbodiimide coupling” does not inherently or explicitly teach the in situ crosslinking at ingestion.

        3. Upper GI-targeted bioadhesion—features not shown or suggested in Nangia.
        Claim 1 also states the microparticle is for “administration of a pharmaceutical composition at the upper gastrointestinal tract.” Nangia repeatedly emphasizes a design that releases or absorbs topiramate in the “lower GI tract” (small intestine, colon) or uses pH ~5.5-7.0 coatings. The concept of an early, localized bioadhesive event in the stomach or proximal duodenum is absent from Nangia.
        In fact, Nangia is actually teaching away from the present invention.
        Nangia, in Co. 12 lines 6-10 states:
        “Applicants have observed that topiramate is unstable under acidic conditions and degrades. Due to its acid-sensitive nature, topiramate may benefit from protection from the relative acidic environment of the upper GI tract, which would otherwise lead to degradation of the drug.”
        And in col. 8, lines 46-48, Nangia states:
        “...coating a topiramate formulation with an enteric coating that reduces or substantially eliminates the release and/or absorption of topiramate in the upper gastrointestinal (GI) tract, such as in the stomach...”
        This explicitly describes a strategy to bypass the stomach and proximal duodenum so that the drug does not substantially release or absorb there. That is the opposite of an in situ crosslinking or bioadhesive event intentionally designed to act in the upper GI region. The applicant's claims teach a microparticle arrangement optimized for administration at or near the upper GI—i.e. where the first coating layer activates (cross-links) upon contact with fluids in the stomach or upper duodenum.
        In summary, because Nangia does not teach or suggest every element of independent claim 1 or its dependent claims 2-11, Nangia fails to anticipate under 35 U.S.C. §102. Applicant therefore respectfully requests withdrawal of the rejection of claims 1-11 under under 35 U.S.C. §102.
        Conclusion
        The present response is intended to be fully responsive to all points of objection raised by the Examiner and is believed to place the claims in condition for allowance. Favorable reconsideration and allowance of the Application is respectfully requested. Should any questions remain unresolved, the Examiner is requested to telephone Applicants' representative at the telephone number or e-mail address below.

    - Office Action 2:
            Notice of Pre-AIA or AIA Status
        1. The present application, filed on or after March 16, 2013, is being examined under the first inventor to file provisions of the AIA.
        2. The following is a quotation of 35 U.S.C. 103 which forms the basis for all obviousness rejections set forth in this Office action:
        A patent for a claimed invention may not be obtained, notwithstanding that the claimed invention is not identically disclosed as set forth in section 102, if the differences between the claimed invention and the prior art are such that the claimed invention as a whole would have been obvious before the effective filing date of the claimed invention to a person having ordinary skill in the art to which the claimed invention pertains. Patentability shall not be negated by the manner in which the invention was made.
        3. Claims 1-13 are rejected under 35 U.S.C. 103 as being unpatentable over GB 2522491 A
        in view of GB 2560026 A. GB 2522491 A discloses a method for forming pellets of a polyhalite by introducing a mixture of a polyhalite and binder to a pan pelletizer. (See pages 2 and 3.) GB 2522491 A teaches on pages 4 and 5 that the polyhalite is crushed and processed to form a polyhalite powder. GB 2522491 A further discloses in the paragraph bridging pages 5 and 6 that the binder may be lignosulphonate or sodium silicate, and in the second full paragraph on page 9 that the material leaving the pan pelletizer is screened into an undersized product, and oversize product. The pelletizing step of GB 2522491 A would constitute a compaction step. The difference between the process disclosed by GB 2522491 A, and that recited in claims 1-8, is that applicant's claims require that the polyhalite is crushed after the compacting step, whereas GB 2522491 A discloses pelletizing after the crushing step. GB 2560026 A discloses a method for forming pellets of polyhalite by treating a polyhalite/starch mixture, and teaches in the last sentence in the paragraph bridging pages 7 and 8 that polyhalite is able to fracture in a number of crystallographic directions when subjected to high compressive forces which produces a fine output efficiency during the HPGR process. It would be obvious from such teaching of GB 2560026 A to modify the process of GB 2522491 A by crushing the polyhalite after the compaction step, since one would appreciate from the disclosure in the last sentence bridging pages 7 and 8 of GB 2560026 A that a compaction of the polyhalite in the process of GB 2522491 A would produce such high compressive forces in the pellets. Regarding claim 3, GB 2522491 A discloses in the last paragraph on page 3 that the evaporite mineral powder may be combined with a liquid. Regarding claim 5, GB
        2522491 A discloses in the paragraph bridging pages 6 and 7 that the binder may include sodium silicate and cement. Regarding claim 8, GB 2560026 A discloses in the first full paragraph on page 10 that the polyhalite/starch mixture is heated to stabilize the mixture. Regarding claim 8, one of ordinary skill in the art would appreciate that mixtures of the binders disclosed in the paragraph bridging pages 5 and 6 could be used. Regarding claims 9-13, GB 2522491 A discloses in the Abstract that the mean diameter of the pellets is 1-8 mm, and GB 2560026 A teaches in the second full paragraph on page 12 that the polyhalite chips have an average diameter of less than about 20 mm and in the fourth paragraph on page 4 that the density ranges from 2.6 to 3.0 g/cm3. it would be expected that the granules would have a single strength of between 0.9 kgForce/granule to 3.3 kgForce/granule, since GB 2560026 A discloses a hardness of around 2.5 to 3.5 in the paragraph bridging pages 4 and 5. Regarding claim 12, it would be within the level of skill of one of ordinary skill in the art to determine a suitable concentration for the lignosulfonate binder.
        4. The following is a quotation of 35 U.S.C. 112(b):
        (b) CONCLUSION.-The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the inventor or a joint inventor regards as the invention.
        The following is a quotation of 35 U.S.C. 112 (pre-AIA), second paragraph:
        The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the applicant regards as his invention.
        5. Claims 1-13 are rejected under 35 U.S.C. 112(b) or 35 U.S.C. 112 (pre-AIA), second paragraph, as being indefinite for failing to particularly point out and distinctly claim the subject matter which the inventor or a joint inventor (or for applications subject to pre-AIA 35 U.S.C. 112, the applicant), regards as the invention. In claim 1, the recitation of "desired" size fine particles is indefinite, since it could not be ascertained whether a given particle size would be "desired". In claims 4 and 10, the recitation of "selected from the group including..." is improper Markush or alternative terminology. Also in claims 4 and 10, the recitation of "and the like" renders the scope of the claims vague and indefinite, since the metes and bounds of binders which might be considered "to be like" those specifically recited would not be ascertainable.
        6. The nonstatutory double patenting rejection is based on a judicially created doctrine grounded in public policy (a policy reflected in the statute) so as to prevent the unjustified or improper timewise extension of the "right to exclude” granted by a patent and to prevent possible harassment by multiple assignees. A nonstatutory double patenting rejection is appropriate where the conflicting claims are not identical, but at least one examined application claim is not patentably distinct from the reference claim(s) because the examined application claim is either

        anticipated by, or would have been obvious over, the reference claim(s). See, e.g., In re Berg, 140 F.3d 1428, 46 USPQ2d 1226 (Fed. Cir. 1998); In re Goodman, 11 F.3d 1046, 29 USPQ2d 2010 (Fed. Cir. 1993); In re Longi, 759 F.2d 887, 225 USPQ 645 (Fed. Cir. 1985); In re Van Ornum, 686 F.2d 937, 214 USPQ 761 (CCPA 1982); In re Vogel, 422 F.2d 438, 164 USPQ 619 (CCPA 1970); In re Thorington, 418 F.2d 528, 163 USPQ 644 (CCPA 1969).
        A timely filed terminal disclaimer in compliance with 37 CFR 1.321(c) or 1.321(d) may be used to overcome an actual or provisional rejection based on nonstatutory double patenting provided the reference application or patent either is shown to be commonly owned with the examined application, or claims an invention made as a result of activities undertaken within the scope of a joint research agreement. See MPEP § 717.02 for applications subject to examination under the first inventor to file provisions of the AIA as explained in MPEP § 2159. See MPEP § 2146 et seq. for applications not subject to examination under the first inventor to file provisions of the AIA. A terminal disclaimer must be signed in compliance with 37 CFR 1.321(b).
        The filing of a terminal disclaimer by itself is not a complete reply to a nonstatutory double patenting (NSDP) rejection. A complete reply requires that the terminal disclaimer be accompanied by a reply requesting reconsideration of the prior Office action. Even where the NSDP rejection is provisional the reply must be complete. See MPEP § 804, subsection I.B.1. For a reply to a non-final Office action, see 37 CFR 1.111(a). For a reply to final Office action, see 37 CFR 1.113(c). A request for reconsideration while not provided for in 37 CFR 1.113(c) may be filed after final for consideration. See MPEP §§ 706.07(e) and 714.13.
        The USPTO Internet website contains terminal disclaimer forms which may be used. Please visit www.uspto.gov/patent/patents-forms. The actual filing date of the application in which the form is filed determines what form (e.g., PTO/SB/25, PTO/SB/26, PTO/AIA/25, or PTO/AIA/26) should be used. A web-based eTerminal Disclaimer may be filled out completely online using web-screens. An eTerminal Disclaimer that meets all requirements is auto-processed and approved immediately upon submission. For more information about eTerminal
        Disclaimers, refer to www.uspto.gov/patents/apply/applying-online/eterminal-disclaimer.
        7. Claims 1-13 are rejected on the ground of nonstatutory double patenting as being unpatentable over claims 1-6 of U.S. Patent No. 10,934,225. Although the claims at issue are not identical, they are not patentably distinct from each other because claim 5 of US 10,934,225 recites that the binder may be inorganic such as lignosulfonate.
        8. CA 2963014 A1 and CA 2963011 A1 are made of record for disclosing the production of sulfate of potash granules using lignosulfonates as a binder.

        9. GB 2530757 A is made of record for disclosing the production of pelletized polyhalite using starch as a binder.
        10. Any inquiry concerning this communication or earlier communications from the examiner should be directed to Wayne Langel whose telephone number is 571-272-1353. The
        examiner can normally be reached Monday through Friday from 8:15 am to 4:15 pm..
        If attempts to reach the examiner by telephone are unsuccessful, the examiner's supervisor, Anthony Zimmer, can be reached on 571-270-3591. The fax phone number for the organization where this application or proceeding is assigned is 571-273-8300.
        Information regarding the status of published or unpublished applications may be obtained from Patent Center. Unpublished application information in Patent Center is available to registered users. To file and manage patent submissions in Patent Center, visit: https:// patentcenter.uspto.gov. Visit https://www.uspto.gov/patents/apply/patent-centerfor
        more information about Patent Center and https://www.uspto.gov/patents/docx for information about filing in DOCX format. For additional questions, contact the Electronic Business Center (EBC) at 866-217-9197 (toll-free). If you would like assistance from a USPTO Customer Service Representative, call 800-786-9199 (IN USA or Canada) or 571-272-1000.

    - Response to Office Action 2:
            AMENDMENT

        Commissioner for Patents
        P.O. Box 1450
        Alexandria, VA 22313-1450

        Sir:
            This communication is responsive to the Office Action mailed on July 17, 2024. Kindly amend the above-referenced patent application as indicated hereinbelow.

            AMENDMENTS TO THE CLAIMS

        Kindly amend the claims as follows:
        1.	(Currently Amended) A process for the compaction of Polyhalite, comprising:
        mixing a feed of polyhalite with an inorganic binder suspension or solution in a mixer to yield a mixture;
        compacting said mixture in a compactor to yield masses;
        crushing said masses in a crusher to yield particles; and
        screening said particles in a screener to yield particles in three different sizes:
        oversized fine particles which undergo a second crushing process and are retuned to said screener for screening,
        undersized fine particles which are transferred to said mixer for further mixing and compacting, and
        desired size fine particles between 1-6 mm, which are transferred to a polish screener for glazing and oiling,
        wherein said process is a wet process and said granule is comprised only of said Polyhalite, said binder and water.

        2.	 (Currently Amended) The process of claim 1, wherein said process is a wet process including the addition of a binder further comprising drying at 150 degrees Celsius.

        3.	(Original) The process of claim 3, wherein said binder is added to the mixer with said feed of polyhalite.

        4.	(Currently Amended) The process of claim 4, wherein said binder is selected from the group consisting of including, bentonite, sodium silicate, lignosulfonates, hydrated lime, bitumen, Portland cement, clay, acids (nitric, hydrochloric, phosphoric, sulphuric), water, water glass, cements, Fly Ash, Potassium and Sodium Silicate, MgO, CaO, Geo-polymers, and the like, or a combination thereof.

        5.	(Original) The process of claim 4, wherein said binder is 2%CaO+5% Na2SiO3.

        6.	(Original) The process of claim 4, wherein said binder is Mg Lignosulfonate.

        7.	(Original) The process of claim 1 wherein said process is a dry process wherein said mixture is heated in a heater after being mixed in said mixer.

        8.	(Original) The process of claim 1, further comprising adding an organic binder to the inorganic binder to create a mixture of organic-inorganic binder.

        9.	(Original) A compacted granule of Polyhalite, comprising an inorganic binder, wherein said granule comprises a thickness of 9 to 33 mm and a specific gravity (S.G.) of between 1.9 to 2.8 g/cm3, a size of  1.4-4.75 mm and a single strength of between 0.9 kgForce/granule to 3.3 kgForce/granule.

        10.	(Currently Amended) The granule of claim 9, wherein said binder is selected from the group consisting of including, bentonite, sodium silicate, lignosulfonates, hydrated lime, bitumen, Portland cement, clay, acids (nitric, hydrochloric, phosphoric, sulphuric), water, water glass, cements, Fly Ash, Potassium and Sodium Silicate, MgO, CaO, Geo-polymers, and the like, or a combination thereof.

        11.	(Original) The granule of claim 10, wherein said binder is 2%CaO+5% Na2SiO3.

        12.	(Original) The granule of claim 10, wherein said binder is Mg Lignosulfonate, in a concentration of between 2% and 5%.

        13.	(Original) The granule of claim 10, wherein said binder further comprises an organic binder, added to the inorganic binder to create a mixture of organic-inorganic binder.

        REMARKS
        The present response is intended to be fully responsive to all points of rejection raised by the Examiner in the instant Office Action and is believed to place the application in condition for allowance. Favorable reconsideration and allowance of the application is respectfully requested.
        Claim Amendments
        Claims 1-13 are currently pending. Claims 1, 2, 4 and 10 have been amended. No new matter has been added.
        Claim Rejections - 35 U.S.C. §103
        Claims 1-13 are rejected under 35 U.S.C. §103 as being unpatentable over GB2522491 in view of GB2560026.
        Applicant draws the Examiner's attention to the fact that the claimed invention is focused on compaction of pure Polyhalite.
        Applicant submits that compaction of polyhalite is a unique challenge. See, for example, page 8, lines 13-16, of the specification as filed, which states (emphasis added):
        “According to some demonstrative embodiments, the present invention allows for the effective compaction of Polyhalite which is difficult to compact under regular conditions due to the difficulty in achieving deformation of Polyhalite particles.”
        The Examiner submits that “The pelletizing step of GB2522491 would constitute a compaction step”, however, Applicant respectfully disagrees.
        GB2522491 discusses the method of granulation of Polyhalite, and specifically uses pelletising methods of granulation (such as using a pan pelletiser). See, for example, the second paragraph on page 2 of GB2522491.
        Applicant submits that pelletising via granulation is a completely different process than that of compaction, and a person of ordinary skill in the art would not consider using the teachings of a document which focuses on granulation for conceiving a compaction method.
        GB2522491 is completely silent with regard to compaction of Polyhalite. In fact, in order to overcome the challenges of granulating Polyhalite,GB2522491 suggests crushing the polyhalite into powder. See, for example, the second paragraph on page 2 of GB2522491.
        In strong contrast, the claimed invention teaches the compaction of pure Polyhalite without any need for modification of the mineral in terms of particle size. Thus, not only does GB2522491 fail to teach or suggest the compaction process of the claimed invention, GB2522491 actually teaches away from the claimed invention.
        GB2560026 cannot cure the deficiencies of GB 2522491 because it is also completely silent with regard to compaction of Polyhalite. The Examiner refers to the last sentences in the paragraph bridging pages 7 and 8 as mentioning that Polyhalite is able to fracture in a number of crystallographic directions when subjected to high compressive forces. From this the Examiner concludes that it may be combined with the teachings of GB2522491 to produce high compressive forces in the pellets (page 3 of the office action).
        Applicant respectfully disagrees and draws the Examiner's attention to the fact that GB2560026 also discusses the creation of Polyhalite powder and does not mention or even remotely suggest a compaction process, yet is focused on granulation.
        The fact that both GB2560026 and GB 2522491 teach that Polyhalite needs to be crushed before handling (let alone when focusing on granulation) only emphasizes the uniqueness of the claimed invention in which compaction takes place without any need for crushing the Polyhalite into powder.
        Accordingly, Applicant submits that GB2560026 and GB 2522491, whether alone or in combination, fail to teach or suggest the limitations of independent claims 1 and 9. Claims 1 and 9 are therefore deemed allowable over the cited references, as are, a fortiori, claims dependent therefrom.
        Reconsideration of claims 1-13 under 35 U.S.C. §103 is therefore respectfully requested.
        Claim Rejections - 35 U.S.C. §112
        Claims 1-13 are rejected under 35 U.S.C. §112 as allegedly being indefinite.
        Claim 1 has been amended to limit the “desired size” particles to between 1-6 mm.
        Claims 4 and 10 have been amended to address the Examiner's comments.
        Reconsideration of claims 1-13 under 35 U.S.C. §112 is therefore respectfully requested.
        Nonstatutory Double Patenting Rejections
        Claims 1-13 are rejected on the ground of nonstatutory double patenting as being unpatentable over claims 1-6 of U.S. Patent No. 10,934,225, for which a terminal disclaimer has been submitted.

        Conclusion
            Applicant respectfully submits that consideration of the above remarks renders the present application in condition for allowance, which action Applicant respectfully solicits.

    # Output Format:
    Use the sendPatentRequest tool to send the output to Make.com. \
    Place each section in the corresponding field in the schema in string format. \
    Additionally, put the same output text in the chat output.
    Make sure that you included all of these sections:
    - Title
    - Amendment
    - Amendments to the specification
    - Amendments to the claims
    - Remarks
    - Conclusion

    After you finish writing a section leave a space of 2 lines before starting to write the next section.

    # Important Rules
    - Always check the files for relevant information before providing an answer.
    - The response should be based on the template that is in the files: 4042US - RESOA.docx,  4052US RESOA.docx and 4838US RESOA.docx. It must comply with the rules and laws of patent requests.
    - If there is insufficient information ask the user clarifying questions in order to obtain all the data you need in order to craft a response to the Office Action. Always ask only one question at a time.
    - Make each section of the response as elaborate as possible.
    - Send each section to make.com using sendPatentRequest tool in corresponding field of the schema, as detailed in the action steps.
    - Make sure you include all the relevant sections.
    - Wait to be prompted to write each section individually. Do NOT write all the response to the Office Action in a single output.


{context}

Question: {question}
Helpful Answer:"""

In [ ]:
prompt_template = PromptTemplate.from_template(prompt_template)

In [ ]:
question = """Respond to this clain rejection:
The following is a quotation of 35 U.S.C. 112(b):
(b) CONCLUSION.-The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the inventor or a joint inventor regards as the invention.
The following is a quotation of 35 U.S.C. 112 (pre-AIA), second paragraph: The specification shall conclude with one or more claims particularly pointing out and distinctly claiming the subject matter which the applicant regards as his invention.
Claim 4 is rejected under 35 U.S.C. 112(b) or 35 U.S.C. 112 (pre-AIA), second paragraph, as being indefinite for failing to particularly point out and distinctly claim the subject matter which the inventor or a joint inventor (or for applications subject to pre- AIA 35 U.S.C. 112, the applicant), regards as the invention.
Claim 4 contains the trademark/trade name Carbopol. Where a trademark or trade name is used in a claim as a limitation to identify or describe a particular material or product, the claim does not comply with the requirements of 35 U.S.C. 112(b) or 35 U.S.C. 112 (pre-AIA), second paragraph. See Ex parte Simpson, 218 USPQ 1020 (Bd. App. 1982). The claim scope is uncertain since the trademark or trade name cannot be used properly to identify any particular material or product. A trademark or trade name is used to identify a source of goods, and not the goods themselves. Thus, a trademark or trade name does not identify or describe the goods associated with the trademark or trade name. In the present case, the trademark/trade name is used to
identify/describe specific polyacrylic polymers and, accordingly, the identification/description is indefinite.
"""

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"],
    "date": lambda x: x["date"]
})

In [ ]:
inputs.invoke({"question": question, "date": "19-02-2025"})

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain = inputs | prompt_template | model | output_parser

In [ ]:
async for t in chain.astream({"question": question, "date": "19-02-2025"}):
    print(t, end='')
    time.sleep(0.1)

## Extracting information from the documents

In [ ]:
OA_loader = PyPDFLoader("/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US - OA - fixed.pdf")
OA_doc = OA_loader.load()
OA_doc

In [ ]:
OA_doc_txt = "\n".join([d.page_content for d in OA_doc])
OA_doc_txt

In [ ]:
# all_dates = model.invoke(f"what dates are in this document: {OA_doc[0].page_content}?").content
mail_date = model.invoke(f"what is the mail date in this document: {OA_doc[0].page_content}? respond in this format: 'Month Day, Year'. As an output give only the date with no leading words.").content
mail_date

In [ ]:
rejected_claims = model.invoke(f"Which claims have been rejected in this document:{OA_doc_txt}").content
rejected_claims

In [ ]:
patent_loader = PyPDFLoader("/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US Application as filed.pdf")
patent_doc = patent_loader.load()
patent_doc

In [ ]:
patent_doc_txt = "\n".join([d.page_content for d in patent_doc])
patent_doc_txt

In [ ]:
original_claims = model.invoke(f"You can see the claims that were rejected here: {rejected_claims}. What are the original claims that were rejected? You can find them here:{patent_doc_txt}. Make the output in list form").content
original_claims

In [ ]:
print(original_claims)

In [ ]:
# loader = Docx2txtLoader("./example_data/fake.docx")

# data = loader.load()

# data

In [ ]:
lean_prompt_template = """
    # Persona
    You are a patent lawyer. You are an expert in responding to Office Actions in a way that complies with the \
    MANUAL OF PATENT EXAMINING PROCEDURE. You have a deep knowledge and understanding in the patent rules and laws, \
    and you know how to respond to claim rejections either by making amendments to the claims or by making \
    remarks to dispute the rejection.

    # Task
    You will get a Patent Application document, and an Office Action document from the examiner of the \
    United States Patent and Trademark Office. Your job is to respond to all claim rejections in the best way \
    possible. You can amend a claim, add a remark, dispute the rejection, remove claim or unite two claims. \
    Always use context retrived from the knowledge-base in order to give the best response. \
    Answer in a formal way that is acceptable by the United States Patent and Trademark Office.

    # Action steps
    - Read the Patent Application document and the Office Action document.
    - Read the claim rejection in the Office Action document.
    - Find the original claims that were rejected in the Patent Application document.
    - Check data retrieved from the knowledge-base.
    - Respond to the claim rejection in the best way possible.


    {context}

    Patent Application document: {patent_application}
    Office Action document: {office_action}
    Helpful Answer:

    """

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["patent_application"] + x["office_action"]),
    "patent_application": lambda x: x["patent_application"],
    "office_action": lambda x: x["office_action"]
})

In [ ]:
inputs.invoke({"patent_application": patent_doc_txt, "office_action": OA_doc_txt})

In [ ]:
lean_prompt_template = PromptTemplate.from_template(lean_prompt_template)

In [ ]:
chain = inputs | lean_prompt_template | model | output_parser

In [ ]:
response = chain.invoke({"patent_application": patent_doc_txt, "office_action": OA_doc_txt})


In [ ]:
print(response)

## Building a graph

### Creating the components

In [ ]:
class State(TypedDict):
    patent_application: str
    office_action: str
    knowledge_base: List[Document]
    rejected_claims: str
    original_claims: str
    strategy: str
    references: List[str]
    other_patents: dict
    responses: List[str]
    draft: str

In [ ]:
def load_documents(state: State):

    print("\nLoading documents ...")
    # Load the patent application
    PA_loader = PyPDFLoader(state['patent_application'])
    PA_docs = PA_loader.load()
    PA_doc_txt = "\n".join([d.page_content for d in PA_docs])

    # Load the office action
    OA_loader = PyPDFLoader(state['office_action'])
    OA_docs = OA_loader.load()
    OA_doc_txt = "\n".join([d.page_content for d in OA_docs])

    print("✅ Documents loaded")

    return {'patent_application': PA_doc_txt, 'office_action': OA_doc_txt}


In [ ]:
patent_application = "/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US Application as filed.pdf"
office_action = "/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US - OA - fixed.pdf"

docs_return = load_documents({'patent_application': patent_application, 'office_action': office_action})
docs_return

In [ ]:
def extract_claims_and_references(state: State):

    print("\nExtracting claims and references ...")

    # Extract the rejected claims
    rejected_claims = model.invoke(f"Which claims have been rejected in this document:{state['office_action']}").content
    print("✅ Extracted rejected claims")

    # Extract the original claims
    original_claims = model.invoke(f"You can see the claims that were rejected here: {rejected_claims}. What are the original claims that were rejected? You can find them here:{state['patent_application']}. Make the output in list form").content
    print("✅ Extracted original claims")

    # Extract the references
    patents_str = model.invoke(f"what other patents are referenced in this document: {state['office_action']}? \
                                Return only a list patent numbers without added text or explanation. \
                                The format of the patent number should be: two letters indicating the country, \
                                a string of numbers, and a final letter. Example: GB2522491A. \
                                All the characters of a patent number should be consecutive with no spaces in between. \
                                Write each patent number on a separate line").content

    patents_list = patents_str.split('\n')
    patents_list = [p.strip() for p in patents_list]
    print("✅ Extracted references")

    return {'rejected_claims': rejected_claims, 'original_claims': original_claims, 'references': patents_list}

In [ ]:
claims_return = extract_claims_and_references({
    'patent_application': docs_return['patent_application'],
    'office_action': docs_return['office_action']
})
claims_return

In [ ]:
print(claims_return['original_claims'])

In [ ]:
def retrieve(state:State):

    print("\nRetrieving knowledge base ...")

    retrieved_docs = vector_store.max_marginal_relevance_search(
        query=state["original_claims"] + '\n' + state["rejected_claims"],
        k=10,
        fetch_k=20
    )
    print("✅ Knowledge base retrieved")

    return {'knowledge_base': retrieved_docs}


In [ ]:
retrieve_return = retrieve({
    'original_claims': claims_return['original_claims'],
    'rejected_claims': claims_return['rejected_claims']
})
retrieve_return

In [ ]:
retrieve_txt = "\n".join([d.page_content for d in retrieve_return['knowledge_base']])
retrieve_txt

In [ ]:
claims_return['references']

In [ ]:
url = "https://serpapi.com/search"
params = {
    'engine': 'google_patents',
    'q': claims_return['references'][5],
    'api_key': f"{SERP_API_KEY}"
}

serpapi_link = requests.get(url, params=params).json()['organic_results'][0]['serpapi_link']
serpapi_link

In [ ]:
claims_return['references'][0]

In [ ]:
params = {
    'engine': 'google_patents_details',
    'patent_id': claims_return['references'][0],
    'api_key': "3ff47b6c86b22d6c0e62c11bd0d89b56ac87e539e57bdea2a64fb0e2cdf2d308"
}

description_link = requests.get(serpapi_link, params=params).json()['description_link']
description_link

In [ ]:
url = "https://api.tavily.com/extract"
headers = {
    "Authorization": f"Bearer {TAVILY_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "urls": description_link,
    "include_images": False,
    "extract_depth": "advanced"
}

response = requests.request("POST", url, json=payload, headers=headers)
data = response.json()

In [ ]:
print(data['results'][0]['raw_content'])

In [ ]:
def get_patent_text(patent_number):

    # Get the patent serpapi link
    url = "https://serpapi.com/search"
    params = {
        'engine': 'google_patents',
        'q': patent_number,
        'api_key': f"{SERP_API_KEY}"
    }

    serpapi_link = requests.get(url, params=params).json()['organic_results'][0]['serpapi_link']


    # Get the patent description link
    params = {
        'engine': 'google_patents_details',
        'patent_id': patent_number,
        'api_key': f"{SERP_API_KEY}"
    }

    description_link = requests.get(serpapi_link, params=params).json()['description_link']


    # Extract the text using tavily
    url = "https://api.tavily.com/extract"
    headers = {
        "Authorization": f"Bearer {TAVILY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "urls": description_link,
        "include_images": False,
        "extract_depth": "advanced"
    }

    response = requests.request("POST", url, json=payload, headers=headers)
    data = response.json()
    content = data['results'][0]['raw_content']

    return content

In [ ]:
get_patent_text(claims_return['references'][1])

In [ ]:
def fetch_patents(state: State):

    print("\nFetching other relevant patents ...")

    other_patents = {}
    if len(state['references']) != 0:
        for pat_num in state['references']:
            content = get_patent_text(pat_num)
            other_patents[pat_num] = content

    print("✅ Fetched other relevant patents")

    return {'other_patents': other_patents}


In [ ]:
fetch_patents_return = fetch_patents({
    'references': claims_return['references']
})
fetch_patents_return

In [ ]:
def decide_action(state: State):

    print("\nDeciding on the best strategy ...")

    # Load the prompt template
    with open("/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/prompt/system_prompt_dev.txt", "r") as f:
        prompt_template = f.read()

    prompt = PromptTemplate.from_template(prompt_template)

    # Concatenating the KB into a single string
    knoeledge_base = "\n".join([d.page_content for d in state['knowledge_base']])

    chain = prompt | model | output_parser

    response = chain.invoke({
        'patent_application': state['patent_application'],
        'office_action': state['office_action'],
        'knowledge_base': knoeledge_base,
        'rejected_claims': state['rejected_claims'],
        'original_claims': state['original_claims']
    })

    print("✅ Strategy decided")

    return {'strategy': response}

In [ ]:
model_return = decide_action({
    'patent_application': docs_return['patent_application'],
    'office_action': docs_return['office_action'],
    'knowledge_base': retrieve_return['knowledge_base'],
    'rejected_claims': claims_return['rejected_claims'],
    'original_claims': claims_return['original_claims']
})
model_return

In [ ]:
print(model_return['strategy'])

In [ ]:
temp_resp = model.invoke(f"how many action steps are in this strategy: {model_return['strategy']}? Return only a number").content
temp_resp

In [ ]:
for i in "":
    print("test")

In [ ]:
for i in range(int(temp_resp)):
    step = model.invoke(f"What is action step number {i+1} in this strategy: {model_return['strategy']}? \
                          Return just the action step as it is, do not add text.").content
    print(f"{i+1}. {step}\n")

    needed_references = model.invoke(f"what patent references are needed for this action step: {step}? \
                                       Return only the patent numbers in a list without any added characters. \
                                       If there are no references needed in this action step, return an empty string").content

    if needed_references:
        needed_references = needed_references.split('\n')
        for ref in needed_references:
            print(ref.strip())
            if ref.strip() in fetch_patents_return['other_patents']:
                references_text = fetch_patents_return['other_patents'][ref.strip()]
                print(references_text)
                print("\n")

    print('\n')

In [ ]:
def execute(state: State):

    print("\nExecuting the strategy ...")

    knowledge_base = "\n".join([d.page_content for d in state['knowledge_base']])

    num_of_steps = model.invoke(f"how many action steps are in this strategy: {state['strategy']}? \
                                  Return only a number").content

    responses = []
    for i in range(int(num_of_steps)):
        step = model.invoke(f"What is action step number {i+1} in this strategy: {model_return['strategy']}? \
                          Return just the action step as it is, do not add text.").content

        needed_references = model.invoke(f"what patent references are needed for this action step: {step}? \
                                       Return only the patent numbers in a list without any added characters. \
                                       If there are no references needed in this action step, return an empty string").content

        if needed_references:
            needed_references = needed_references.split('\n')
            references_text = ""
            for ref in needed_references:
                if ref.strip() in state['references']:
                    references_text = references_text + state['other_patents'][ref.strip()] + '\n'

        # Load the prompt template
        with open("/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/prompt/system_prompt_exec.txt", "r") as f:
            exec_prompt_temp = f.read()

        exec_prompt = PromptTemplate.from_template(exec_prompt_temp)

        chain = exec_prompt | model | output_parser

        response = chain.invoke({
            'knowledge_base': knowledge_base,
            'patent_application': state['patent_application'],
            'strategy': state['strategy'],
            'references_text': references_text,
            'step': step
        })

        responses.append(response)

    print("✅ Strategy executed")

    return {'responses': responses}

In [ ]:
execute_return = execute({
    'knowledge_base': retrieve_return['knowledge_base'],
    'patent_application': docs_return['patent_application'],
    'strategy': model_return['strategy'],
    'references': claims_return['references'],
    'other_patents': fetch_patents_return['other_patents'],
})
execute_return

In [ ]:
print(execute_return['responses'][1])

In [ ]:
def edit_draft(state: State):

    print("Preparing the draft ... \n\n")

    responses = "\n".join(state['responses'])

    example_loader = PyPDFLoader("/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US RESOA 2.10.pdf")
    example_docs = example_loader.load()
    example_txt = "\n".join([d.page_content for d in example_docs])

    edit_prompt_template = f"You are an expert patent lawyer. Your job is to craft a formal response to the Office Action \
        in a way that is compliant with the MANUAL OF PATENT EXAMINING PROCEDURE. You will get the response in raw format. \
        Here is the raw response: {responses}. You need to edit the response to make it formal and professional. \
        Use this example to see how the structure should look like: {example_txt}. \
        Make the response to the Office Action professional, elamorate and complete."

    edit_system_prompt = PromptTemplate.from_template(edit_prompt_template)
    chain = edit_system_prompt | model | output_parser

    draft = chain.invoke({'responses': responses, 'example_txt': example_txt}, config={"streaming": True})

    return {'draft': draft}

In [ ]:
edit_return = edit_draft({'responses': execute_return['responses']})
edit_return

In [ ]:
print(edit_return['draft'])

### Conecting it all together

In [ ]:
len('')

In [ ]:
patents_list = ''.split('\n')
patents_list

In [ ]:
patents_list = [p.strip() for p in patents_list if len(p.strip()) > 1]
patents_list

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
graph_builder.add_node("load_documents", load_documents)
graph_builder.add_node("extract_claims_and_references", extract_claims_and_references)
graph_builder.add_node("fetch_patents", fetch_patents)
graph_builder.add_node("retrieve", retrieve)

graph_builder.add_node("decide_action", decide_action)
graph_builder.add_node("execute", execute)
graph_builder.add_node("edit_draft", edit_draft)

In [ ]:
graph_builder.set_entry_point("load_documents")

graph_builder.add_edge("load_documents", "extract_claims_and_references")
graph_builder.add_edge("extract_claims_and_references", "fetch_patents")
graph_builder.add_edge("fetch_patents", "retrieve")

graph_builder.add_edge("retrieve", "decide_action")
graph_builder.add_edge("decide_action", "execute")
graph_builder.add_edge("execute", "edit_draft")

graph_builder.add_edge("edit_draft", END)

In [ ]:
# Add memory, compile and add a thread to the config
memory = MemorySaver()
patent_graph = graph_builder.compile(checkpointer=memory)

In [ ]:
patent_graph

In [ ]:
thread = {"configurable": {"thread_id": "1"}}

In [ ]:
patent_application = "/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US Application as filed.pdf"
office_action = "/Users/Mark/code/MarkBerkovics/projects/Lawyer_bot/docs/4838US/4838US - OA - fixed.pdf"

In [ ]:
for m in patent_graph.stream({'patent_application': patent_application, 'office_action': office_action}, config=thread, stream_mode='messages'):
    if isinstance(m[0], langchain_core.messages.ai.AIMessageChunk):
        if "streaming" in m[1].keys():
            print(m[0].content, end='', flush=True)
            time.sleep(0.1)

## Send the draft to a google doc